# Taschenrechner

## Teil 3) Taschenrechner für komplexe Zahlen mit User-Eingabe
Implementieren Sie einen Taschenrechner für komplexe Zahlen:
- Eingabe von komplexen Zahlen
- Zugriff auf alle Rechenoperationen
- Visualisierungen der Zahlen und Rechnungen mittels Diagramme
<br>

Konzipieren Sie eine geeignete Nutzerführung mittels Textausgabe und Textkommandos. Geben Sie aussagekräftige Fehlermeldungen bei Bedienfehlern

In [14]:
class KomplexeZahl():
    def __init__(self,real=0,imaginaer=0):
        self.real=real
        self.imaginaer=imaginaer
    def __add__(self,other):
        if(type(other)==KomplexeZahl):
            return KomplexeZahl(self.real+other.real,self.imaginaer+other.imaginaer)
        elif(type(other)==int or type(other)==float):
            return KomplexeZahl(self.real+other,self.imaginaer)
        else:
            raise TypeError
    def __sub__(self,other):
        if(type(other)==KomplexeZahl):
            return KomplexeZahl(self.real-other.real,self.imaginaer-other.imaginaer)
        elif(type(other)==int or type(other)==float):
           return KomplexeZahl(self.real-other,self.imaginaer)
        else:
            raise TypeError
    def __mul__(self,other):
        if(type(other)==KomplexeZahl):
            return KomplexeZahl(self.real*other.real-self.imaginaer*other.imaginaer,
                                self.real*other.imaginaer+self.imaginaer*other.real)
        elif(type(other)==int or type(other)==float):
            return KomplexeZahl(self.real*other,self.imaginaer*other)
        else:
            raise TypeError
    def __truediv__(self,other):
        if(type(other)==KomplexeZahl):
            if other.real == 0 and other.imag == 0:
                raise ComplexNumberError("Division durch Null ist nicht definiert für komplexe Zahlen.")
            div = other.real**2 + other.imag**2
            new_real = (self.real * other.real + self.imag * other.imag) / div
            new_imag = (self.imag * other.real - self.real * other.imag) / div
            return ComplexNumber(new_real, new_imag)
        elif(type(other)==int or type(other)==float):
            return KomplexeZahl(self.real/other,self.imaginaer/other)
        else:
            raise TypeError
    def __radd__(self,other):
        if(type(other)==int or type(other)==float):
            return KomplexeZahl(self.real+other,self.imaginaer)
        else:
            raise TypeError
    def __rsub__(self,other):
        if(type(other)==int or type(other)==float):
            return KomplexeZahl(other-self.real,-self.imaginaer)
        else:
            raise TypeError
    def __rmul__(self,other):
        if(type(other)==int or type(other)==float):
            return KomplexeZahl(self.real*other,self.imaginaer*other)
        else:
            raise TypeError
    def __rtruediv__(self,other):
        if(type(other)==int or type(other)==float):
            return KomplexeZahl(other,0) / self
        else:
            raise TypeError

    
    def __pow__(self,other):
        num=self
        for i in range(int(other-1)):
            num*=self
        return num

    def __str__(self):
        return (f"{self.real if self.real!=0 else ''} {'{0:{1}}i'.format(self.imaginaer, '+' if self.real else '') if self.imaginaer!=0 else ''}") #{'+' if self.real!=0 else ''}

In [15]:
def Eingabe(Satz):
    print(Satz)
    #später die komplexe Zahl
    komplex=None
    
    #solange die Eingabe falsch ist wiederholen
    while(komplex==None):

        #Eingabe des Nutzers
        Eingabe=input()

        #Leerzeichen aus der Eingabe entfernen
        stripedeingabe=''
        for i in Eingabe:
            stripedeingabe=stripedeingabe+( i if i!=' ' else '')

        #komplex wird gesetzt, falls die Eingabe richtig war,
        #sonst wird die Schleife wiederholt
        komplex=ChecktheString(stripedeingabe.strip())

        #rückgabe der Zahl
        return komplex

In [16]:
import re
import math
def ChecktheString(string):
#string="2pi+isin(2pi)"
#fehler string="2"
    #string="i"
    Checkchars(string)
               
    string=Checkdouble(string)

    string=re.sub(r"sin","Sin",string)

    Check_i_sin_cos(string)

    string=Checkpi(string)
    
    if(not Checkbrackets(string)):
        print("klammern sind falsch gesetzt")
        raise Exception
    
    string=StandartizeI(string)
    
    komplex=Rechnen(string)
    #komplex ist die zahl
    return komplex

In [17]:
def Checkchars(string):
    #auf falsche Zeichen prüfen
    checkchars=re.compile(r"[0-9sSiInpPcCo\(\)\+\*\-\/\.\^]")
    for i in string:
        if(re.search(checkchars,i)==None):
            print("falsche Zeichen benutzt",i)
            raise Exception

In [18]:
def Checkdouble(string):
    
    string=re.sub(r"(?:\+\-)|(?:\-\+)","-",string)

    string=re.sub(r"(?:\+\+)|(?:\-\-)","+",string)
    return string

In [19]:
def Check_i_sin_cos(string):
    #auf i,s,p,c,o prüfen
    checki=re.compile(r".*[iI].*[iI].*")
    checksin=re.compile(r"[sS]in\((.+?)\)")
    checkpi=re.compile(r"[pP]i")
    checkcos=re.compile(r"[cC]os\((.+?)\)")
    istring=re.sub(checksin,'€',string)
    istring=re.sub(checkpi,'_',istring)
    istring=re.sub(checkcos,'&',istring)
    if(re.search(r"[sSpPcCo]",istring)!=None):
        print("s,p,c,o falsch benutzt")
        raise Exception


In [20]:
def Checkpi(string):
    #pi normieren und auflösen
    prepi=re.search(r"([0-9])[pP]i",string)
    while(prepi!=None):
        string=re.sub(r"[0-9][pP]i",(prepi.group(1)+"*pi"),string)
        prepi=re.search(r"([0-9])[pP]i",string)
    
    postpi=re.search(r"[pP]i([0-9])",string)
    while(postpi!=None):
        string=re.sub(r"[pP]i[0-9]",("pi*"+postpi.group(1)),string)
        postpi=re.search(r"[pP]i([0-9])",string)
    string=re.sub(r"[pP]i","3.14159265",string)
    return string

In [21]:
def Checkbrackets(string):
    #auf klammern prüfen
    klammern=0
    for i in string:
        if(i=='('):
            klammern+=1
        elif(i==')'):
            klammern-=1
    if(klammern!=0):
        return False
    return True

In [22]:
#
#ersetzt falsche sachen, da alle gleich ersetzt werden
def StandartizeI(string):
    #i standartisieren
    checksti=re.compile(r"([0-9\)])[iI](?![n])")
    getsti=1
    while(getsti!=None):
        getsti=re.search(checksti,string)
        if(getsti!=None):
            string=re.sub(checksti,f'{getsti.group(1)}*i',string,count=1)
    checksti2=re.compile(r"(?<![sScC])[iI]([0-9\(ScsC])")
    getsti2=1
    while(getsti2!=None):
        getsti2=re.search(checksti2,string)
        if(getsti2!=None):
            string=re.sub(checksti2,f'i*{getsti2.group(1)}',string,count=1)
    #nicht sicher ob es wichtig ist
    checkonlyi=re.compile(r"(\*)?[iI](?(1)(?:\*)?|\*)")
    if(re.search(checkonlyi,string)==None):
        string=re.sub(r"[iI](?![sn])",'1*i',string)
    return string

In [23]:
def Rechnen(str):
    if(str[0]=="+" or str[0]=="-"):
        str="0"+str
    rechn=None
    if(str=="i"):
        return KomplexeZahl(0,1)
    if(re.search(r"^\(.+\)$",str)!=None):
        return Rechnen(str[1:-1])
    if(re.search(r"^[sS]in\(.+\)$",str)!=None):
        print(Rechnen(str[4:-1]))
        return math.sin(Rechnen(str[4:-1]))
    if(re.search(r"^[Cc]os\(.+\)$",str)!=None):
        return math.cos(Rechnen(str[4:-1]))
        
    str=re.sub(r"(?<=[\*\/\^])[\+]","&",str)
    str=re.sub(r"(?<=[\*\/\^])[\-]","€",str)
    while True:
        erg=re.split(r"[\+\-](?!.*[\+\-])",str,maxsplit=1)
        rechn=re.search(r"[\+\-](?!.*[\+\-])",str)
        if(len(erg)==2):
            ch=False
            for i in erg:
                if(not Checkbrackets(i)):
                    str=erg[0]+('&' if rechn.group(0)=="+" else '€')+erg[1]
                    ch=True
            if(ch):
                continue
        break
    #beide seiten auf klammern prüfen und falls wiederholen
    #checkbrackets()
    str=re.sub(r"\&","+",str)
    str=re.sub(r"\€","-",str)
    erg[0]=re.sub(r"\&","+",erg[0])
    erg[0]=re.sub(r"\€","-",erg[0])
    if(len(erg)==2):
        erg[1]=re.sub(r"\&","+",erg[1])
        erg[1]=re.sub(r"\€","-",erg[1])
        erg[0]=Rechnen(erg[0])
        erg[1]=Rechnen(erg[1])
    else:
        while True:
            tmp=erg[0]
            erg=re.split(r"[\*\/](?!.*[\*\/])",erg[0],maxsplit=1)
            rechn=re.search(r"[\*\/](?!.*[\*\/])",tmp)
            if(len(erg)==2):
                ch=False
                for i in erg:
                    if(not Checkbrackets(i)):
                        str=erg[0]+('&' if rechn.group(0)=="*" else '€')+erg[1]
                        ch=True
                if(ch):
                    erg=[str]
                    continue
                    
            break
        str=re.sub(r"\&","*",str)
        str=re.sub(r"\€","/",str)
        erg[0]=re.sub(r"\&","*",erg[0])
        erg[0]=re.sub(r"\€","/",erg[0])
        if(len(erg)==2):
            erg[1]=re.sub(r"\&","*",erg[1])
            erg[1]=re.sub(r"\€","/",erg[1])
            erg[0]=Rechnen(erg[0])
            erg[1]=Rechnen(erg[1])
        else:
            erg=re.split(r"\^",erg[0],maxsplit=1)
            if(len(erg)==2):
                erg=Rechnen(erg[0])**Rechnen(erg[1])
            else:
                return float(erg[0])
            '''tmperg=float(erg[0])
            del(erg[0])
            for i in erg:
                tmperg=float(math.pow(tmperg,float(i)))
            erg=tmperg
            '''
    #print("erg",erg,"rechn",rechn,"str",str)
    if(type(erg) in (float,KomplexeZahl)):
        return erg
    else:
        rechn=rechn.group(0)
        if(rechn=="+"):
            return (erg[0]+erg[1])
        elif(rechn=="-"):
            return (erg[0]-erg[1])
        elif(rechn=="*"):
            return (erg[0]*erg[1])
        elif(rechn=="/"):
            if(erg[1]==0):
                print("Durch Null teilen ist nicht möglich")
                raise Exception
            return (erg[0]/erg[1])
        else:
            return erg[0]
            #....
     #formel mit (i) als komplexer zahl
     #besitzt keine klammern,sin,cos,pi,freie i
     #kein ^ mit i
     #wichtig - am anfang ist möglich

In [24]:
try:
    print(Eingabe("Welchen Term möchtest du auflösen?: "))
except:
    print("Term konnte nicht aufgelöst werden")
#2/3*(-4^-2+-sin(5)i)
# -0.0417 0.639
#2.5*(cos(4)+sin(3)i)
#-1.63 + 0.35
# 2+(2-3+2i)-2
#(2+2i)^2
#3i(4+-3/5i)
#1i+2i*-4+4/7*(i+2+-2/4I)Sin(3pi)

Welchen Term möchtest du auflösen?: 


 3i(4+-3/5i)


1.7999999999999998 +12.0i
